# Import

In [62]:
%pip install tensorflow keras


In [63]:
%pip install keras-tuner


In [87]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
%pip install hmmlearn
%pip install pgmpy

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [66]:
%pip install keras-nlp --upgrade

Note: you may need to restart the kernel to use updated packages.


In [67]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
import keras_tuner as kt
from tensorflow.keras.models import load_model
import keras_nlp

In [68]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score

In [69]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
import hmmlearn.hmm
from hmmlearn.hmm import GaussianHMM
from sklearn_crfsuite import CRF
from sklearn.metrics import log_loss, hinge_loss, precision_score, recall_score, f1_score, roc_auc_score

In [70]:
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination

In [71]:
## Options
pd.set_option("max_colwidth", None)

In [72]:
# Get the absolute path to the 'src' directory
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
sys.path.append(project_root)
print(project_root)

e:\2_LEARNING_BKU\2_File_2\K22_HK242\CO3117_Machine_Learning\Main


In [73]:
from src.features.build_features_utils import *  # Assuming build_features_utils is inside build_features.py
from src.models.models_utils import *  # Assuming utils.py exists inside src/models/

# Dict

In [ ]:
# Dictionary for models
MODEL_DICT = {
    "decision_tree": DecisionTreeClassifier,
    "perceptron": Perceptron,
    "mlp": MLPClassifier,
    "bayesian": GaussianNB,
    "random_forest": RandomForestClassifier,
    "xgboost": xgb.XGBClassifier,
    "logistic_regression": LogisticRegression,
    
    "svm": SVC
} 

# Dictionary for model parameters
MODEL_PARAMS = {
    # "decision_tree": {
    #     "criterion": ["gini", "entropy"],
    #     "max_depth": [10, 20],
    #     "min_samples_split": [2, 5],
    #     "min_samples_leaf": [1, 2],
    #     "max_features": ["sqrt", "log2"]
    # },
    
    "decision_tree": {
        "criterion": ["gini", "entropy"],
        "max_depth": [10, 20, 30, 40],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
        "max_features": ["sqrt", "log2"]
    },
    
    # "perceptron": {
    #     "max_iter": [1000, 2000],
    #     "tol": [1e-3],
    #     "eta0": [0.001],
    #     "penalty": ["l2"],
    #     "alpha": [0.0001, 0.001]
    # },
    
    "perceptron": {
        "max_iter": [1000, 2000],
        "tol": [1e-3, 1e-4],
        "eta0": [0.001, 0.01, 0.1],
        "penalty": [None, "l2", "l1"],
        "alpha": [0.0001, 0.001, 0.01]
    },
    
    "mlp": {
        "hidden_layer_sizes": [(100,)],
        "activation": ["tanh", "logistic"],
        "solver": ["sgd"],
        "alpha": [0.01],
        "batch_size": [32],
        "max_iter": [2000],
    },
    
    # "mlp": {
    #     "hidden_layer_sizes": [(50,), (100,), (50, 50), (100, 100)],
    #     "activation": ["relu", "tanh", "logistic"],
    #     "solver": ["adam", "sgd"],
    #     "alpha": [0.0001, 0.001, 0.01],
    #     "batch_size": [32, 64, 128],
    #     "max_iter": [500, 1000],
    #     "learning_rate": ["constant", "invscaling", "adaptive"]
    # },
    
    "bayesian": {
        "priors": [None, [0.5, 0.5], [0.4, 0.6], [0.3, 0.7], [0.2, 0.8], [0.1, 0.9], [0.05, 0.95]],
        "var_smoothing": [1e-9, 1e-8, 1e-7]
    },
    
    "random_forest": {
        "n_estimators": [100, 200],
        "max_depth": [10],
        "min_samples_split": [2, 5],
        "min_samples_leaf": [1, 2],
        "max_features": ["sqrt", "log2"],
        "bootstrap": [True, False]
    },
    
    # "random_forest": {
    #     "n_estimators": [50, 100, 200],
    #     "max_depth": [None, 10, 20, 30],
    #     "min_samples_split": [2, 5, 10],
    #     "min_samples_leaf": [1, 2, 4],
    #     "max_features": ["auto", "sqrt", "log2"],
    #     "bootstrap": [True, False]
    # },
    
    "xgboost": {
        "n_estimators": [100],
        "learning_rate": [0.01, 0.1],
        "max_depth": [6, 10]
    },
    
    # "xgboost": {
    #     "n_estimators": [100, 200, 300],
    #     "learning_rate": [0.01, 0.1, 0.2],
    #     "max_depth": [3, 6, 10],
    #     "subsample": [0.8, 1.0],
    #     "colsample_bytree": [0.8, 1.0],
    #     "gamma": [0, 0.1, 0.2]
    # },
    
    "svm": {
        "kernel": ["linear"],
        "C": [0.001, 0.01, 0.1, 1],
        "gamma": [0.1, 0.01, "scale", "auto"]
    },
    
    # "svm": {
    #     "kernel": ["linear", "rbf", "poly"],
    #     "C": [0.1, 1, 10, 100],
    #     "gamma": [0.1, 0.01, "scale", "auto"],
    #     "degree": [2, 3, 4]
    # },
    
    # "logistic_regression": {
    #     "penalty": ["l2"],
    #     "C": [0.1, 1.0],
    #     "max_iter": [1000, 2000]
    # },
    
    "logistic_regression": {
        "penalty": ["l1", "l2", "elasticnet", None],
        "C": [0.1, 1.0, 10.0],
        "max_iter": [1000, 2000]
    },
    
    
    # "hmm": {
    #     "n_components": [2],  # Keep it small
    #     "covariance_type": ["diag"],  # Simpler covariance type
    #     "n_iter": [500],  # Reduce iterations
    #     "init_params": ["stmc"],  # Initialize start probabilities, transition matrix, and means/covariance
    #     "params": ["stmc"]
    # },
    
    "hmm": {
        "n_components": [2, 3, 4],
        "covariance_type": ["diag", "full", "tied"],
        "n_iter": [100, 200],
        "init_params": ["c", "s", "cs"],
        "params": ["c", "t", "ct"]
    },
    
    "bayes_network": {
        "structure": [None],
        "n_bins": [2],
        "strategy": ["kmeans"],
        "min_unique_values": [2],
        "max_features": [10]
    },
    
    # "crf": {
    #     "c1": [0.1, 0.01],  # L1 Regularization
    #     "c2": [0.1, 0.01],  # L2 Regularization
    #     "max_iterations": [50, 100]  # Limit iterations
    # }
}

BEST_MODEL_PARAMS = {
    "decision_tree": {
        "criterion": ["gini"],
        "max_depth": [40],
        "min_samples_split": [10],
        "min_samples_leaf": [4],
        "max_features": ["sqrt"]
    },
    
    "perceptron": {
        "max_iter": [1000],
        "tol": [1e-3],
        "eta0": [0.001],
        "penalty": ["l2"],
        "alpha": [0.0001]
    },
    
    "mlp": {
        "hidden_layer_sizes": [(100,)],
        "activation": ["tanh", "logistic"],
        "solver": ["sgd"],
        "alpha": [0.01],
        "batch_size": [32],
        "max_iter": [2000],
    },
    
    "bayesian": {
        "priors": [[0.3, 0.7]],
        "var_smoothing": [1e-9]
    },
    
    "random_forest": {
        "n_estimators": [100],
        "max_depth": [10],
        "min_samples_split": [5],
        "min_samples_leaf": [1],
        "max_features": ["sqrt"]
    },
    
    "xgboost": {
        "n_estimators": [150],
        "learning_rate": [0.1],
        "max_depth": [15]
    },
    
    "svm": {
        "kernel": ["linear"],
        "C": [0.001, 0.01, 0.1, 1],
        "gamma": [0.1, 0.01, "scale", "auto"]
    },
    
    "logistic_regression": {
        "penalty": ["l2"],
        "C": [0.1],
        "max_iter": [1000]
    },
    
    "hmm": {
        "n_components": [2, 3, 4],
        "covariance_type": ["diag", "full", "tied"],
        "n_iter": [100, 200],
        "init_params": ["c", "s", "cs"],
        "params": ["c", "t", "ct"]
    },
    
    "bayes_network": {
        "structure": [None],
        "n_bins": [2],
        "strategy": ["kmeans"],
        "min_unique_values": [2],
        "max_features": [10]
    },
    
    # "crf": {
    #     "c1": [0.1, 0.01],  # L1 Regularization
    #     "c2": [0.1, 0.01],  # L2 Regularization
    #     "max_iterations": [50, 100]  # Limit iterations
    # }
}

# Dictionary for dimensionality reduction methods
DIMENSIONALITY_REDUCTION_DICT = {
    "pca": PCA,
    "lda": LDA,
}

# Load dataset

In [75]:
# Load dataset
dataset_path = os.path.join(project_root, "data", "final", "final_clean_no_neutral_no_duplicates_v1.csv")
df = pd.read_csv(dataset_path)


In [76]:
df.head()

,target,text,text_clean,text_length,text_clean_length
0,0.0,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D",switchfoot awww thats bummer shoulda got david carr third day,19,10
1,0.0,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!,upset cant update facebook texting might cry result school today also blah,21,12
2,0.0,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds,kenichan dived many times ball managed save rest go bounds,18,10
3,0.0,my whole body feels itchy and like its on fire,whole body feels itchy like fire,10,6
4,0.0,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there.",nationwideclass behaving im mad cant see,21,6


In [77]:
# Replace target 4 with 1
df["target"] = df["target"].replace(4, 1)


# Build features

## Defined

In [78]:
feature_methods = ["tfidf", "count", "word2vec", "glove"]
df_sampled = df.sample(n=1000, random_state=42)

In [79]:
doc_lst = df_sampled["text_clean"].tolist()
label_lst = df_sampled["target"].tolist()

In [80]:
X_train_features_dict, X_test_features_dict, y_train, y_test = build_vector_for_text(df_sampled, feature_methods, project_root)


🔎 Running feature extraction...



Feature Extraction Progress:   0%|          | 0/4 [00:00<?, ?it/s]


🔍 Processing feature extraction using: tfidf...
✅ tfidf - Train shape: (800, 2000), Test shape: (200, 2000)


Feature Extraction Progress:  25%|██▌       | 1/4 [00:00<00:00,  5.71it/s]


🔍 Processing feature extraction using: count...


...

✅ count - Train shape: (800, 2000), Test shape: (200, 2000)

🔍 Processing feature extraction using: word2vec...


INFO:gensim.models.keyedvectors:loading projection weights from C:\Users\User/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from C:\\Users\\User/gensim-data\\word2vec-google-news-300\\word2vec-google-news-300.gz', 'binary': True, 'encoding': 'utf8', 'datetime': '2025-03-01T11:11:37.585444', 'gensim': '4.3.3', 'python': '3.12.8 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:48:34) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'load_word2vec_format'}
Feature Extraction Progress:  75%|███████▌  | 3/4 [02:19<01:03, 63.40s/it]

✅ word2vec - Train shape: (800, 300), Test shape: (200, 300)

🔍 Processing feature extraction using: glove...


INFO:gensim.models.keyedvectors:loading projection weights from C:\Users\User/gensim-data\glove-wiki-gigaword-100\glove-wiki-gigaword-100.gz
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (400000, 100) matrix of type float32 from C:\\Users\\User/gensim-data\\glove-wiki-gigaword-100\\glove-wiki-gigaword-100.gz', 'binary': False, 'encoding': 'utf8', 'datetime': '2025-03-01T11:14:18.746341', 'gensim': '4.3.3', 'python': '3.12.8 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:48:34) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'load_word2vec_format'}
Feature Extraction Progress: 100%|██████████| 4/4 [05:00<00:00, 75.14s/it] 


✅ glove - Train shape: (800, 100), Test shape: (200, 100)


In [81]:
print("\n📊 Dataset Shapes:")

# Print the shape of feature matrices for each feature method
for feature_method, X_train in X_train_features_dict.items():
    print(f"🔹 X_train ({feature_method}): {X_train.shape}")
    
for feature_method, X_test in X_test_features_dict.items():
    print(f"🔹 X_test ({feature_method}): {X_test.shape}")

# Print y_train and y_test shapes
print(f"\n🎯 y_train shape: {y_train.shape}")
print(f"🎯 y_test shape: {y_test.shape}")



📊 Dataset Shapes:
🔹 X_train (tfidf): (800, 2000)
🔹 X_train (count): (800, 2000)
🔹 X_train (word2vec): (800, 300)
🔹 X_train (glove): (800, 100)
🔹 X_test (tfidf): (200, 2000)
🔹 X_test (count): (200, 2000)
🔹 X_test (word2vec): (200, 300)
🔹 X_test (glove): (200, 100)

🎯 y_train shape: (800,)
🎯 y_test shape: (200,)


## Part nay dang test

In [89]:
X_train_pca_dict, X_test_pca_dict, y_train, y_test = build_vector_for_text(df_sampled, feature_methods, project_root, "pca", 100)


🔎 Running feature extraction...



Feature Extraction Progress:   0%|          | 0/4 [00:00<?, ?it/s]


🔍 Processing feature extraction using: tfidf...


Feature Extraction Progress:  25%|██▌       | 1/4 [00:03<00:10,  3.43s/it]

✅ tfidf - Train shape: (800, 100), Test shape: (200, 100)

🔍 Processing feature extraction using: count...


...

✅ count - Train shape: (800, 100), Test shape: (200, 100)

🔍 Processing feature extraction using: word2vec...


INFO:gensim.models.keyedvectors:loading projection weights from C:\Users\User/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from C:\\Users\\User/gensim-data\\word2vec-google-news-300\\word2vec-google-news-300.gz', 'binary': True, 'encoding': 'utf8', 'datetime': '2025-03-01T11:41:06.453684', 'gensim': '4.3.3', 'python': '3.12.8 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:48:34) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'load_word2vec_format'}
Feature Extraction Progress:  75%|███████▌  | 3/4 [02:14<00:59, 59.71s/it]

✅ word2vec - Train shape: (800, 300), Test shape: (200, 300)

🔍 Processing feature extraction using: glove...


INFO:gensim.models.keyedvectors:loading projection weights from C:\Users\User/gensim-data\glove-wiki-gigaword-100\glove-wiki-gigaword-100.gz
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (400000, 100) matrix of type float32 from C:\\Users\\User/gensim-data\\glove-wiki-gigaword-100\\glove-wiki-gigaword-100.gz', 'binary': False, 'encoding': 'utf8', 'datetime': '2025-03-01T11:42:21.253975', 'gensim': '4.3.3', 'python': '3.12.8 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:48:34) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'load_word2vec_format'}
Feature Extraction Progress: 100%|██████████| 4/4 [03:29<00:00, 52.46s/it]

✅ glove - Train shape: (800, 100), Test shape: (200, 100)


In [90]:
print("\n📊 Dataset Shapes:")

# Print the shape of feature matrices for each feature method
for feature_method, X_train in X_train_pca_dict.items():
    print(f"🔹 X_train ({feature_method}): {X_train.shape}")
    
for feature_method, X_test in X_test_pca_dict.items():
    print(f"🔹 X_test ({feature_method}): {X_test.shape}")

# Print y_train and y_test shapes
print(f"\n🎯 y_train shape: {y_train.shape}")
print(f"🎯 y_test shape: {y_test.shape}")



📊 Dataset Shapes:
🔹 X_train (tfidf): (800, 100)
🔹 X_train (count): (800, 100)
🔹 X_train (word2vec): (800, 300)
🔹 X_train (glove): (800, 100)
🔹 X_test (tfidf): (200, 100)
🔹 X_test (count): (200, 100)
🔹 X_test (word2vec): (200, 300)
🔹 X_test (glove): (200, 100)

🎯 y_train shape: (800,)
🎯 y_test shape: (200,)


In [ ]:
# X_train_lda_dict, X_test_lda_dict, y_train, y_test = build_vector_for_text(df_sampled, feature_methods, project_root, "lda", 100)

In [ ]:
# print("\n📊 Dataset Shapes:")

# # Print the shape of feature matrices for each feature method
# for feature_method, X_train in X_train_lda_dict.items():
#     print(f"🔹 X_train ({feature_method}): {X_train.shape}")
    
# for feature_method, X_test in X_test_lda_dict.items():
#     print(f"🔹 X_test ({feature_method}): {X_test.shape}")

# # Print y_train and y_test shapes
# print(f"\n🎯 y_train shape: {y_train.shape}")
# print(f"🎯 y_test shape: {y_test.shape}")


# New API Call

In [ ]:
model_name_lst = [
    "decision_tree",
    "random_forest", 
    "xgboost", 
    "perceptron", 
    "mlp", 
    "lstm",
    "bayesian",
    "GA",
    "hmm",
    "bayesnet",
    "logistic_regression",
    "svm"
]

In [ ]:
trained_model = os.path.join(project_root, "src", "models")

In [ ]:
# %pip uninstall tf-nightly
# %pip install tensorflow


In [ ]:
# %pip show keras-nlp


In [ ]:
train_general_model(df_sampled, doc_lst, label_lst, model_name_lst, feature_methods, MODEL_DICT, MODEL_PARAMS, X_train_features_dict, X_test_features_dict, y_train, y_test)

In [ ]:
predict_general_model(model_name_lst, feature_methods, X_test_features_dict, y_test, trained_model)